# Determining the percentage of reference allele counts in Pf v6 samples

## Pfv6 Data Release

The MalariaGEN Plamodium falciparum Community project recently released version 6 of sequencing data (Oct 10, 2019). This data release contains 7,113 genomes from 73 study sites. Samples from the Pf3k project for which sequence alignments have been downloaded are included. Updates include upgraded variant discovery and genotype calling pipeline. For more details on the data release and available content, please reference the project website: https://www.malariagen.net/resource/26. 

Variant data have been provided in VCF and Zarr formats.The newer Zarr format is faster and is better for parallelization. The downside is files can only be used in Python and is less documented for use cases, but follows HDF5 conventions. I anticipate any work outside of Python (mainly R) will use VCF files directly mitigating this issue.   

## Goals

The goal of this analysis is to replicate earlier work by Josh Protoctor on the Pf3k v3 release on the heterozygosity at variant sites. A higher proportion of heterozygozity scores across the genome within a sample is indicative of polyinfections.


# Approach 1: Parsing the variant VCF files
My first attempt to calculate the reference allele count from the VCFs, which included subsetting the SNPs of interest (same parameters discussed below), and then converting the allele counts from wide to long then doing simple division betwen two columns with Bash. I wrapped these commands into a Snakemake pipeline. 

This worked - files were created (with --letency wait warnings), but everytime I restarted the command to create a file that was missing it would rerun all the files. I predict this has to do with opening a new Linux subsystem checking files stored on Windows/Dropbox. Using --touch to update the .snakemake master file with new locations claimed the missing files existed. I am not confident the produced files are completed since --rerun-incomplete flagged all of the steps. Followed-up with creating subset VCF files that will be needed for other analyses on the command line for safety:

# Approach 2: Parsing the variant Zarr file

The scikit-allel package will be used to parse the variant file. I am following the tutorial available from Alistair Miles  http://alimanfoo.github.io/2018/04/09/selecting-variants.html (Accessed Oct. 2019).  

In [1]:
# load packages - have been installed in vcf virtual env
import os, sys
import zarr
import numpy as np
import pandas as pd
import allel  
import dask.array as da

In [2]:
# read in zarr files
zarr_path='/mnt/md0/malariaGen_genomic/pf_v6/Pf_6.zarr.zip'
callset = zarr.open(zarr_path, mode='r')
callset.tree(expand=True)

/
 ├── calldata
 │   ├── AD (6051696, 7113, 7) int16
 │   ├── DP (6051696, 7113) int16
 │   ├── GT (6051696, 7113, 2) int8
 │   ├── PGT (6051696, 7113) object
 │   └── PID (6051696, 7113) object
 ├── samples (7113,) object
 └── variants
     ├── AC (6051696, 6) int32
     ├── ALT (6051696, 6) object
     ├── AN (6051696,) int32
     ├── CDS (6051696,) bool
     ├── CHROM (6051696,) object
     ├── DP (6051696,) int32
     ├── FILTER_Apicoplast (6051696,) bool
     ├── FILTER_Centromere (6051696,) bool
     ├── FILTER_InternalHypervariable (6051696,) bool
     ├── FILTER_Low_VQSLOD (6051696,) bool
     ├── FILTER_Mitochondrion (6051696,) bool
     ├── FILTER_PASS (6051696,) bool
     ├── FILTER_SubtelomericHypervariable (6051696,) bool
     ├── FILTER_SubtelomericRepeat (6051696,) bool
     ├── ID (6051696,) object
     ├── POS (6051696,) int32
     ├── QUAL (6051696,) float32
     ├── REF (6051696,) object
     ├── RegionType (6051696,) object
     ├── SNPEFF_AMINO_ACID_CHANGE (6051696,) object
     ├── SNPEFF_CODON_CHANGE (6051696,) object
     ├── SNPEFF_EFFECT (6051696,) object
     ├── SNPEFF_EXON_ID (6051696,) object
     ├── SNPEFF_FUNCTIONAL_CLASS (6051696,) object
     ├── SNPEFF_GENE_NAME (6051696,) object
     ├── SNPEFF_IMPACT (6051696,) object
     ├── SNPEFF_TRANSCRIPT_ID (6051696,) object
     ├── VQSLOD (6051696,) float32
     ├── altlen (6051696, 6) int32
     ├── is_snp (6051696,) bool
     └── numalt (6051696,) int32

## Identify high quality samples and variants for subsetting 


### Samples
Removal of replicate, low coverage, mislabelled, and mixed species samples resulted in the inclusion of 5,970 of the 7,113 samples in the study. The sampels that failed the above QC metrics have been flagged in the metadata file as 'QC pass' with explanation in 'Exlusion reason' if applicable.

In [3]:
# read in metadata
pf_meta = '/mnt/md0/malariaGen_genomic/pf_v6/Pf_6_samples.txt'
meta = pd.read_csv(pf_meta, sep='\t', header=0)
meta.columns = [x.replace(' ', '_') for x in meta.columns]
# check names in metadata are the same as the variant file
samples = callset['samples'][:]
print(np.all(samples == meta['Sample'].values))

True


In [4]:
# assign metadata sample names the same number as the entry in th variant file for subsetting
samples_list = list(samples)
meta['callset_index'] =  [samples_list.index(s) for s in meta['Sample']]

In [5]:
# keep the samples that pass qc metrics
meta_filt = meta[meta.Exclusion_reason == 'Analysis_set']
sample_subset = meta_filt.callset_index.values
# check it matches the consotrium provided number
len(sample_subset) == 5970

True

### Variants
It is also recommended to only consider bi-allelic coding SNPs in the core genome with VSQLOD > 6. The VSQLOD metric measures the probability of a variant being false. There should be 83,168 SNPs that meet the strict criteria. 

In [6]:
quality_set = callset['variants/FILTER_PASS'][:]
snp_set = callset['variants/is_snp'][:]
vsqlod_set = callset['variants/VQSLOD'][:]  > 6
biallelic_set = callset['variants/numalt'][:] == 1
variant_keep = quality_set & snp_set & vsqlod_set & biallelic_set
filt_n = np.count_nonzero(variant_keep)
print("Number of filtered genes:", filt_n)
print("Difference of filtered genes from consortia:", filt_n - 83168)

Number of filtered genes: 89578
Difference of filtered genes from consortia: 6410


The number of SNPs that match the criteria was orthoganlly checked with the VCF files using the same filtering parameters provided by the authors of the release notes. Added CDS=1 and change >6 to >=6 (see reasoning below). 

We have ~6k more variants that should be keept according the the consortia... hmm. The bcftools command for subsetting also include retaining coding positions. Does filtering noncoding positions give us the correct number?  
After meeting Roberto Amato at ASTMH and connecting us with Richard Pearson at Sanger. With a few updates I am able to get the right number with additionalyl including VSQLOD >= 6 instead of just > 6. 

In [13]:
vsqlod_set = callset['variants/VQSLOD'][:]  >= 6
# core_set = callset['variants/CDS'][:] == 1 
core_set = callset['variants/CDS'][:]
variant_keep = quality_set & snp_set & vsqlod_set & biallelic_set & core_set
filt_n = np.count_nonzero(variant_keep)
print("Number of filtered genes:", filt_n)
print("Difference of filtered genes from consortia:", filt_n - 83168)

Number of filtered genes: 83168
Difference of filtered genes from consortia: 0


Coding regions put us about ~1k SNPs below the consortia estimate. Perhaps the additional SNPs are found in the mitochondrial and apicoplast variant files that are not merged in the Zarr object?


Nope, nonautosomal variants are not a source of the missing variants. 

Coding region SNPs have higher biological constraint, so let's start with those sites.  Pruning non-segregating sites across populations will also improve the informativeness of the SNPs in regards to selection. For each SNP we can count the copies of the reference allele (0) and each of the alternate alleles (1, 2, 3) observed in each population.

First, map the sample indixes to population names. 

In [117]:
meta_filt.Population.value_counts()

WAF     2231
ESEA    1262
WSEA    1079
EAF      739
CAF      344
OCE      201
SAS       77
SAM       37
Name: Population, dtype: int64

In [8]:
subpops = {
    'all': list(meta_filt.callset_index),
    'WAF': meta_filt[meta_filt.Population == 'WAF'].callset_index.tolist(),
    'EAF': meta_filt[meta_filt.Population == 'EAF'].callset_index.tolist(),
    'CAF': meta_filt[meta_filt.Population == 'CAF'].callset_index.tolist(),
    'ESEA': meta_filt[meta_filt.Population == 'ESEA'].callset_index.tolist(),
    'WSEA': meta_filt[meta_filt.Population == 'WSEA'].callset_index.tolist(),
    'OCE': meta_filt[meta_filt.Population == 'OCE'].callset_index.tolist(),
    'SAS': meta_filt[meta_filt.Population == 'SAS'].callset_index.tolist(),
    'SAM': meta_filt[meta_filt.Population == 'SAM'].callset_index.tolist()
}

In [9]:
gt_zarr = callset['calldata/GT']
gt_dask = allel.GenotypeDaskArray(gt_zarr)
gt_daskSub = gt_dask.subset(variant_keep).compute()
gt_daskSub

<GenotypeArray shape=(89578, 7113, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/1 0/0 0/1 1/1 1/1 ... 1/1 1/1 1/1 0/0 1/1

In [143]:
ac_subpops = gt_daskSub.count_alleles_subpops(subpops)
is_seg = ac_subpops['all'].is_segregating()[:]
populations = meta_filt.Population.unique()
for pop in populations:
    print(pop, len(ac_subpops[pop].is_segregating()[:]),
          np.count_nonzero(ac_subpops[pop].is_segregating()[:])

WAF 82193 57105 57105
EAF 82193 28710 28710
WSEA 82193 14806 14806
ESEA 82193 14439 14439
OCE 82193 11326 11326
SAM 82193 6013 6013
SAS 82193 12613 12613
CAF 82193 20765 20765


In [140]:
populations = meta_filt.Population.unique()
for pop in populations:
    print(pop, len(ac_subpops[pop].is_singleton(allele=1)[:]),
          np.count_nonzero(ac_subpops[pop].is_singleton(allele=1)[:]),
          np.count_nonzero(ac_subpops[pop].is_doubleton(allele=1)[:]))

WAF 82193 4510 22613
EAF 82193 2424 7901
WSEA 82193 441 1097
ESEA 82193 283 917
OCE 82193 481 866
SAM 82193 5 450
SAS 82193 1158 1786
CAF 82193 2480 4802


allel.locate_unlinked(

## Subset Zarr objects

To calculate the percentage of reads that align to the reference, we need to extract the number of reads that align to that position and the number of reads that align to the reference allele. The number of reads at each position are found in the 'DP' field, while a individual counts per allele are in the 'AD' field, comma seprated by counts to each allele in order of reference + n(alleles) in REF. 

In [20]:
# pull total number of high quality reads that align to each position per sample
dp_zarr = callset['calldata/DP']
dp_dask = allel.AlleleCountsDaskArray(dp_zarr)
dp_variant_selection = dp_dask.compress(variant_subset, axis=0).compute()
dp_variant_selection = dp_variant_selection.take(sample_subset, axis=1)

In [21]:
# get the number of reads that align to each of the alleles
ad_zarr=callset['calldata/AD']
ad_array=da.from_zarr(ad_zarr)
ad_variant_selection=ad_array[variant_subset]

In [22]:
# pull the first item in the allele count array to get the reference count
ad_array_ref=ad_variant_selection[:,sample_subset,0]
ad_array_ref=ad_array_ref.compute()

In [23]:
# check both data frames have the same dimension
np.shape(ad_array_ref) == np.shape(dp_variant_selection)

True

In [24]:
# ignore zero division for site where one population may have an allele but other samples may not have adequate coverage
np.seterr(divide='ignore', invalid='ignore')
ref_het_calc=ad_array_ref/dp_variant_selection

## Save to data frame 

In [25]:
# assign the row names that correspond to the chromosome and position
pos_keep = np.where(variant_subset)[0]
chrom = ["chr"+x.split("_",2)[1] for x in callset['variants/CHROM'][:][pos_keep].tolist()]
pos = callset['variants/POS'][:][pos_keep].tolist()
pos_index = [x + "_" + str(y) for x, y in zip(chrom, pos)]

het_df=pd.DataFrame(data=ref_het_calc,
                     index=pos_index,
                     columns=[callset['samples'][i] for i in sample_subset])
het_df
het_df.to_csv('/mnt/c/Users/jribado/Desktop/pf_v6/biAllelicPassSeg_hetCounts.txt', header=True, index=True, sep="\t")
count_df=pd.DataFrame(data=dp_variant_selection[0:,0:],
                     index=pos_index,
                     columns=[callset['samples'][i] for i in sample_subset])
count_df.to_csv('/mnt/c/Users/jribado/Desktop/pf_v6/biAllelicPassSeg_baseCounts.txt', header=True, index=True, sep="\t")